In [1]:
# for "2. Data Loading"
import pandas as pd

# for "3-1. Feature Generation"
import numpy as np

# for "3-2. Feature Engineering"
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import RobustScaler, StandardScaler

# for "4. Modeling with Pycaret"
from pycaret.regression import *

# for "5. Modeling with CatBoostRegressor"
from catboost import CatBoostRegressor
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold

### 1121
추가 사항 석식계 예측에 자기계발 열을 제거하여 학습 진행

### 1122
추가 사항 독립변수 스케일링 진행

In [2]:
# 데이터 로드
pre_tr = pd.read_csv('data_preprocess/pre_tr_1118.csv',encoding = "cp949")
pre_te = pd.read_csv('data_preprocess/pre_te_1118.csv',encoding = "cp949")
pre_tr = pre_tr.set_index("일자")
pre_te = pre_te.set_index("일자")
today = "1122_4"

In [3]:
# 데이터 확인
pre_tr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1205 entries, 2016-02-01 to 2021-01-26
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일      1205 non-null   int64  
 1   정원수     1205 non-null   int64  
 2   휴가자수    1205 non-null   int64  
 3   출장자수    1205 non-null   int64  
 4   야근자수    1205 non-null   int64  
 5   재택근무자수  1205 non-null   float64
 6   조식메뉴    1205 non-null   object 
 7   중식메뉴    1205 non-null   object 
 8   석식메뉴    1205 non-null   object 
 9   중식계     1205 non-null   float64
 10  석식계     1205 non-null   float64
 11  출근자수    1205 non-null   float64
 12  월       1205 non-null   int64  
 13  년도      1205 non-null   int64  
dtypes: float64(4), int64(7), object(3)
memory usage: 141.2+ KB


조식메뉴, 중식메뉴, 석식메뉴 다 없어도 될거 같은데,,,?
일자도 일단 제외하고 진행

In [4]:
#중식 타겟 데이터
y_lun = pre_tr["중식계"]
# 석식 타겟 데이터
y_din = pre_tr["석식계"]

## 중식계 예측

In [5]:
train_dr = pre_tr.loc[:,"조식메뉴":"석식계"]
X_train = pre_tr.drop(train_dr,axis=1)

In [6]:
# minmax scale 활용
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
col_na = X_train.loc[:,  "요일":"년도"].columns
col_na
X_train.loc[:, "요일":"년도"] = minmax.fit_transform(
    X_train[col_na])

In [7]:
X_train["중식계"] = pre_tr["중식계"]
display(X_train)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도,중식계
일자,,,,,,,,,,
2016-02-01,0.00,0.000000,0.022481,0.323442,0.227969,0.000000,0.664300,0.090909,0.0,1039.0
2016-02-02,0.25,0.000000,0.022481,0.391691,0.305556,0.000000,0.649451,0.090909,0.0,867.0
2016-02-03,0.50,0.000000,0.027477,0.412463,0.106322,0.000000,0.641059,0.090909,0.0,1017.0
2016-02-04,0.75,0.000000,0.067444,0.531157,0.340038,0.000000,0.584248,0.090909,0.0,978.0
2016-02-05,1.00,0.000000,0.212323,0.415430,0.032567,0.000000,0.497095,0.090909,0.0,925.0
...,...,...,...,...,...,...,...,...,...,...
2021-01-20,0.50,0.542614,0.043297,0.465875,0.003831,0.733583,0.611362,0.000000,1.0,1093.0
2021-01-21,0.75,0.542614,0.057452,0.563798,0.442529,0.658537,0.604906,0.000000,1.0,832.0
2021-01-22,1.00,0.542614,0.193172,0.614243,0.000958,0.568480,0.519690,0.000000,1.0,579.0


In [8]:
reg = setup(X_train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = '중식계', # 목표 변수는 중식계 이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = False, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
            numeric_features=list(X_train.drop(columns = ['중식계']).columns), # 모든 변수가 숫자로써의 의미가 있다.
            session_id = 2021,
            fold_shuffle = True
            )

,Description,Value
0,session_id,2021
1,Target,중식계
2,Original Data,"(1205, 10)"
3,Missing Values,False
4,Numeric Features,9
5,Categorical Features,0
6,Transformed Train Set,"(1203, 9)"
7,Transformed Test Set,"(2, 9)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [9]:
top5 = compare_models(n_select = 5, sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,64.8464,7581.0875,86.6848,0.8243,0.1060,0.0781,0.5170
gbr,Gradient Boosting Regressor,66.4847,8145.4710,89.7528,0.8109,0.1102,0.0806,0.0350
et,Extra Trees Regressor,69.8119,9094.6202,94.7995,0.7888,0.1174,0.0850,0.0730
lightgbm,Light Gradient Boosting Machine,70.1996,8766.0373,93.0467,0.7959,0.1144,0.0850,0.1340
rf,Random Forest Regressor,70.5119,8982.2683,94.2247,0.7912,0.1164,0.0857,0.0930
xgboost,Extreme Gradient Boosting,72.7939,9006.1631,94.5303,0.7905,0.1160,0.0878,0.3210
knn,K Neighbors Regressor,77.9272,10773.6355,103.4344,0.7494,0.1272,0.0947,0.0060
ada,AdaBoost Regressor,84.2602,11669.9176,107.7997,0.7288,0.1333,0.1040,0.0280
huber,Huber Regressor,86.2202,12848.7029,112.8959,0.7036,0.1367,0.1035,0.0080
ridge,Ridge Regression,86.6774,12762.4373,112.5509,0.7054,0.1357,0.1037,0.0050


In [10]:
models = []
for m in top5:
    models.append(tune_model(m, 
                             optimize = 'MAE', 
                             choose_better = True,
                            n_iter = 30))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,78.9488,11025.8190,105.0039,0.7110,0.1265,0.0963
1,63.0393,6529.5830,80.8058,0.8675,0.1049,0.0799
2,68.2747,7892.5475,88.8400,0.8364,0.1225,0.0888
3,71.7355,9495.1475,97.4430,0.7825,0.1135,0.0836
4,78.1814,10873.3472,104.2753,0.7404,0.1305,0.0957
5,73.5592,9813.5537,99.0634,0.7934,0.1199,0.0866
6,61.1499,6165.2312,78.5190,0.8393,0.0983,0.0761
7,66.4448,7422.9547,86.1566,0.8211,0.1025,0.0776
8,64.5410,7572.4043,87.0196,0.8072,0.1055,0.0781
9,72.7832,9994.3565,99.9718,0.7880,0.1273,0.0904


In [11]:
test_dr = pre_te.loc[:,"조식메뉴":"석식메뉴"]
X_test = pre_te.drop(test_dr,axis=1)
X_test.loc[:, "요일":"년도"] = minmax.transform(
    X_test[col_na])
display(X_test)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도
일자,,,,,,,,,
2021-01-27,0.50,0.542614,0.054122,0.418398,0.004789,0.671670,0.664300,0.000000,1.0
2021-01-28,0.75,0.542614,0.067444,0.507418,0.391762,0.652908,0.649451,0.000000,1.0
2021-01-29,1.00,0.542614,0.205662,0.617211,0.000000,0.551595,0.641059,0.000000,1.0
2021-02-01,0.00,0.458807,0.070774,0.335312,0.515326,0.604128,0.584248,0.090909,1.0
2021-02-02,0.25,0.458807,0.032473,0.430267,0.435824,0.589118,0.497095,0.090909,1.0
2021-02-03,0.50,0.458807,0.029975,0.468843,0.004789,0.536585,0.453841,0.090909,1.0
2021-02-04,0.75,0.458807,0.031640,0.504451,0.455939,0.540338,0.441575,0.090909,1.0
2021-02-05,1.00,0.458807,0.121565,0.626113,0.000000,0.480300,0.605552,0.090909,1.0
2021-02-08,0.00,0.458807,0.054122,0.394659,0.660920,0.617261,0.594577,0.090909,1.0


In [12]:
voting = blend_models(models, optimize = 'MAE')
voting = tune_model(voting, 
                 optimize = 'MAE', 
                 choose_better = True,
                 n_iter = 30)

voting = finalize_model(voting)
sample = pd.read_csv('sub/sample_submission.csv', encoding = "cp949")
layer1_pred = voting.predict(X_test)
sample['중식계'] = layer1_pred

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,72.8388,10267.0866,101.3266,0.7309,0.1202,0.0880
1,59.9044,6152.1068,78.4354,0.8752,0.1049,0.0768
2,61.6994,6522.7028,80.7633,0.8648,0.1122,0.0799
3,64.6963,7852.3951,88.6137,0.8201,0.1035,0.0757
4,70.3112,9156.4517,95.6893,0.7814,0.1180,0.0859
5,68.1596,8496.4598,92.1762,0.8211,0.1123,0.0804
6,58.1637,5535.5719,74.4014,0.8557,0.0954,0.0725
7,62.5610,6495.8059,80.5966,0.8435,0.0929,0.0716
8,58.3189,6266.6045,79.1619,0.8405,0.0921,0.0688
9,67.1897,8993.7334,94.8353,0.8092,0.1210,0.0837


## 석식계 예측

In [13]:
train_dr = pre_tr.loc[:,"조식메뉴":"석식계"]
X_train = pre_tr.drop(train_dr,axis=1)
X_train.loc[:, "요일":"년도"] = minmax.fit_transform(
    X_train[col_na])
X_train["석식계"] = pre_tr["석식계"]
display(X_train)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도,석식계
일자,,,,,,,,,,
2016-02-01,0.00,0.000000,0.022481,0.323442,0.227969,0.000000,0.664300,0.090909,0.0,331.0
2016-02-02,0.25,0.000000,0.022481,0.391691,0.305556,0.000000,0.649451,0.090909,0.0,560.0
2016-02-03,0.50,0.000000,0.027477,0.412463,0.106322,0.000000,0.641059,0.090909,0.0,573.0
2016-02-04,0.75,0.000000,0.067444,0.531157,0.340038,0.000000,0.584248,0.090909,0.0,525.0
2016-02-05,1.00,0.000000,0.212323,0.415430,0.032567,0.000000,0.497095,0.090909,0.0,330.0
...,...,...,...,...,...,...,...,...,...,...
2021-01-20,0.50,0.542614,0.043297,0.465875,0.003831,0.733583,0.611362,0.000000,1.0,421.0
2021-01-21,0.75,0.542614,0.057452,0.563798,0.442529,0.658537,0.604906,0.000000,1.0,353.0
2021-01-22,1.00,0.542614,0.193172,0.614243,0.000958,0.568480,0.519690,0.000000,1.0,217.0


In [14]:
# 자기계발 날 제거 
# (석식계 0인 이상치가 발견되는 열들 제거 하여 학습 데이터 생성) 
# 총 43개 열 제거
day_of_dinner0 = X_train.loc[X_train["석식계"] == 0]
display(day_of_dinner0)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도,석식계
일자,,,,,,,,,,
2016-11-30,0.50,0.125000,0.037469,0.492582,0.000000,0.000000,0.672692,0.909091,0.0,0.0
2016-12-28,0.50,0.147727,0.119067,0.545994,0.000000,0.000000,0.608134,1.000000,0.0,0.0
2017-01-25,0.50,0.136364,0.046628,0.480712,0.000000,0.000000,0.673338,0.000000,0.2,0.0
2017-02-22,0.50,0.044034,0.043297,0.626113,0.000000,0.000000,0.602324,0.090909,0.2,0.0
2017-03-22,0.50,0.036932,0.024979,0.575668,0.000000,0.000000,0.624274,0.181818,0.2,0.0
2017-04-26,0.50,0.035511,0.018318,0.780415,0.000000,0.000000,0.584248,0.272727,0.2,0.0
2017-05-31,0.50,0.051136,0.016653,0.664688,0.000000,0.000000,0.617818,0.363636,0.2,0.0
2017-06-28,0.50,0.066761,0.029142,0.646884,0.000000,0.000000,0.619109,0.454545,0.2,0.0
2017-07-26,0.50,0.338068,0.192340,0.608309,0.000000,0.000000,0.624274,0.545455,0.2,0.0


In [15]:
# 자기계발 날 제거 
# (석식계 0인 이상치가 발견되는 열들 제거 하여 학습 데이터 생성) 
# 총 43개 열 제거
X_train = X_train.drop(day_of_dinner0.index,axis=0)
display(X_train)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도,석식계
일자,,,,,,,,,,
2016-02-01,0.00,0.000000,0.022481,0.323442,0.227969,0.000000,0.664300,0.090909,0.0,331.0
2016-02-02,0.25,0.000000,0.022481,0.391691,0.305556,0.000000,0.649451,0.090909,0.0,560.0
2016-02-03,0.50,0.000000,0.027477,0.412463,0.106322,0.000000,0.641059,0.090909,0.0,573.0
2016-02-04,0.75,0.000000,0.067444,0.531157,0.340038,0.000000,0.584248,0.090909,0.0,525.0
2016-02-05,1.00,0.000000,0.212323,0.415430,0.032567,0.000000,0.497095,0.090909,0.0,330.0
...,...,...,...,...,...,...,...,...,...,...
2021-01-20,0.50,0.542614,0.043297,0.465875,0.003831,0.733583,0.611362,0.000000,1.0,421.0
2021-01-21,0.75,0.542614,0.057452,0.563798,0.442529,0.658537,0.604906,0.000000,1.0,353.0
2021-01-22,1.00,0.542614,0.193172,0.614243,0.000958,0.568480,0.519690,0.000000,1.0,217.0


In [16]:
reg = setup(X_train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = '석식계', # 목표 변수는 석식계 이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = False, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
            numeric_features=list(X_train.drop(columns = ['석식계']).columns), # 모든 변수가 숫자로써의 의미가 있다.
            session_id = 2021,
            fold_shuffle = True
            )

,Description,Value
0,session_id,2021
1,Target,석식계
2,Original Data,"(1162, 10)"
3,Missing Values,False
4,Numeric Features,9
5,Categorical Features,0
6,Transformed Train Set,"(1160, 9)"
7,Transformed Test Set,"(2, 9)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [17]:
top5 = compare_models(n_select = 5, sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,43.4490,3449.0813,58.3909,0.7042,0.1428,0.1035,0.5240
gbr,Gradient Boosting Regressor,44.7387,3641.5811,59.9117,0.6895,0.1420,0.1046,0.0350
lightgbm,Light Gradient Boosting Machine,45.5691,3674.3429,60.3644,0.6842,0.1453,0.1073,0.0490
et,Extra Trees Regressor,45.5735,3888.5400,61.6965,0.6672,0.1486,0.1080,0.0710
rf,Random Forest Regressor,46.1801,3868.4921,61.6484,0.6688,0.1479,0.1089,0.0940
xgboost,Extreme Gradient Boosting,47.5574,4116.3284,63.7448,0.6482,0.1518,0.1110,0.2160
knn,K Neighbors Regressor,49.5348,4747.9953,68.4121,0.5928,0.1707,0.1221,0.0080
huber,Huber Regressor,52.9822,5060.5944,70.7205,0.5676,0.1715,0.1278,0.0090
ridge,Ridge Regression,53.2363,5020.3391,70.4493,0.5712,0.1699,0.1275,0.0050
br,Bayesian Ridge,53.2689,5016.0328,70.4184,0.5715,0.1693,0.1273,0.0050


In [18]:
models = []
for m in top5:
    models.append(tune_model(m, 
                             optimize = 'MAE', 
                             choose_better = True,
                            n_iter = 30))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,46.8508,4419.9640,66.4828,0.6343,0.1768,0.1206
1,50.9008,5259.9787,72.5257,0.5015,0.1398,0.1023
2,51.2241,4215.6270,64.9279,0.6139,0.1778,0.1325
3,48.4149,3961.9919,62.9444,0.6550,0.1500,0.1139
4,50.9293,5640.7763,75.1051,0.5936,0.1881,0.1290
5,43.4578,3220.5553,56.7499,0.7310,0.1341,0.1018
6,43.2092,3243.9050,56.9553,0.7238,0.1446,0.1062
7,43.9324,3401.5981,58.3232,0.7401,0.1625,0.1148
8,40.0687,2647.0238,51.4492,0.6824,0.1120,0.0885
9,50.4904,4970.5503,70.5021,0.6284,0.1814,0.1280


In [19]:
test_dr = pre_te.loc[:,"조식메뉴":"석식메뉴"]
X_test = pre_te.drop(test_dr,axis=1)
X_test.loc[:,  "요일":"년도"] = minmax.transform(
    X_test[col_na])
display(X_test)

,요일,정원수,휴가자수,출장자수,야근자수,재택근무자수,출근자수,월,년도
일자,,,,,,,,,
2021-01-27,0.50,0.542614,0.054122,0.418398,0.004789,0.671670,0.664300,0.000000,1.0
2021-01-28,0.75,0.542614,0.067444,0.507418,0.391762,0.652908,0.649451,0.000000,1.0
2021-01-29,1.00,0.542614,0.205662,0.617211,0.000000,0.551595,0.641059,0.000000,1.0
2021-02-01,0.00,0.458807,0.070774,0.335312,0.515326,0.604128,0.584248,0.090909,1.0
2021-02-02,0.25,0.458807,0.032473,0.430267,0.435824,0.589118,0.497095,0.090909,1.0
2021-02-03,0.50,0.458807,0.029975,0.468843,0.004789,0.536585,0.453841,0.090909,1.0
2021-02-04,0.75,0.458807,0.031640,0.504451,0.455939,0.540338,0.441575,0.090909,1.0
2021-02-05,1.00,0.458807,0.121565,0.626113,0.000000,0.480300,0.605552,0.090909,1.0
2021-02-08,0.00,0.458807,0.054122,0.394659,0.660920,0.617261,0.594577,0.090909,1.0


In [20]:
voting = blend_models(models, optimize = 'MAE')
voting = tune_model(voting, 
                 optimize = 'MAE', 
                 choose_better = True,
                 n_iter = 30)

voting = finalize_model(voting)
layer1_pred = voting.predict(X_test)
sample['석식계'] = layer1_pred

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,44.5349,3830.6892,61.8926,0.6830,0.1652,0.1129
1,50.2843,4925.7492,70.1837,0.5331,0.1375,0.1019
2,45.4726,3272.1422,57.2026,0.7003,0.1492,0.1136
3,43.3588,3138.3611,56.0211,0.7267,0.1288,0.0986
4,46.4401,4754.1630,68.9504,0.6575,0.1752,0.1174
5,39.2389,2683.9157,51.8065,0.7758,0.1168,0.0880
6,37.3786,2459.8151,49.5965,0.7906,0.1190,0.0883
7,42.2648,2978.8464,54.5788,0.7724,0.1510,0.1085
8,38.4416,2343.1630,48.4062,0.7189,0.1042,0.0845
9,43.1164,3795.9665,61.6114,0.7162,0.1594,0.1085


In [21]:
sample

,일자,중식계,석식계
0,2021-01-27,1015.248763,386.006092
1,2021-01-28,935.156816,395.421187
2,2021-01-29,627.762992,279.572845
3,2021-02-01,1197.140135,544.541336
4,2021-02-02,965.302636,453.645632
5,2021-02-03,922.009973,398.152626
6,2021-02-04,892.967347,450.080575
7,2021-02-05,657.765397,352.422853
8,2021-02-08,1233.564635,631.710606
9,2021-02-09,1023.842213,530.654817


In [22]:
sample.to_csv('sub/Model_Cat_{}.csv'.format(today), index=False)